* [About this Dataset](#about)
* [EDA](#eda)
* [Data preparation](#dataprep)
* [Modeling](#modeling)
    * [Meta Learners](#metalearners)
        * [T-Learners](#tlearners)
        * [S-Learners](#slearners)
    * [Uplift Trees](#utrees)
* [Conclusions](#conclusions)

# About this Dataset <a class="anchor" id="about"></a>

This is notebook is based on this notebook: https://www.kaggle.com/code/max398434434/marketing-campaign-uplift-modeling
  The dataset was created by The Criteo AI Lab .The dataset consists of 13M rows, each one representing a user with 12 features, a treatment indicator and 2 binary labels (visits and conversions). Positive labels mean the user visited/converted on the advertiser website during the test period (2 weeks). The global treatment ratio is 84.6%. It is usual that advertisers keep only a small control population as it costs them in potential revenue.  
  
Following is a detailed description of the features:  
  
- f0, f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11: feature values (dense, float)
- treatment: treatment group (1 = treated, 0 = control)
- conversion: whether a conversion occured for this user (binary, label)
- visit: whether a visit occured for this user (binary, label)
- exposure: treatment effect, whether the user has been effectively exposed (binary)

In [3]:
#!pip install causalml

In [4]:
#!pip install scikit-uplift

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklift.metrics import uplift_at_k, uplift_auc_score, qini_auc_score, weighted_average_uplift
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel, TwoModels
from sklearn.base import clone
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from IPython.display import Image
from causalml.metrics import *

In [6]:
df = pd.read_csv('../input/uplift-modeling/criteo-uplift-v2.1.csv')
df.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,conversion,visit,exposure
0,12.616365,10.059654,8.976429,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0
1,12.616365,10.059654,9.002689,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0
2,12.616365,10.059654,8.964775,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0
3,12.616365,10.059654,9.002801,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0
4,12.616365,10.059654,9.037999,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,13.190056,5.300375,-0.168679,1,0,0,0


# EDA <a class="anchor" id="eda"></a>

In [7]:
def share_pivot(df, index_cols):
    pivot = pd.DataFrame(df[index_cols].value_counts()).sort_index()
    pivot.columns = ['#']
    pivot['share'] = np.round(pivot['#'] / len(df), 2)
    display(pivot)

In [8]:
share_pivot(df = df, index_cols = ['treatment', 'conversion', 'visit', 'exposure'])

#  share
treatment conversion visit exposure                 
0         0          0     0          2016832   0.14
                     1     0            76042   0.01
          1          1     0             4063   0.00
1         0          0     0         11055129   0.79
                           1           250702   0.02
                     1     0           385634   0.03
                           1           154479   0.01
          1          1     0            13680   0.00
                           1            23031   0.00

In [9]:
share_pivot(df = df, index_cols = ['treatment', 'exposure'])

#  share
treatment exposure                 
0         0          2096937   0.15
1         0         11454443   0.82
          1           428212   0.03

In [10]:
share_pivot(df = df, index_cols = ['treatment', 'conversion'])

#  share
treatment conversion                 
0         0            2092874   0.15
          1               4063   0.00
1         0           11845944   0.85
          1              36711   0.00

In [11]:
share_pivot(df = df, index_cols = ['treatment', 'visit'])

#  share
treatment visit                 
0         0       2016832   0.14
          1         80105   0.01
1         0      11305831   0.81
          1        576824   0.04

# Data preparation <a class="anchor" id="dataprep"></a>

In [12]:
train, test = train_test_split(
    df.drop(columns = ['visit', 'exposure']),# focus on conversion
    test_size = 0.4, 
    random_state = 1
)

In [13]:
class DataUndersampler:
    """Makes dataframe balances by target columns within treatment groups"""
    
    def __init__(self, target_column):
        self.__target_column = target_column
    
    def __make_equal(self, df):
        target_0_count = np.sum(df[self.__target_column] == 0)
        target_1_count = np.sum(df[self.__target_column] == 1)
        
        if target_0_count == target_1_count:
            pass
        elif target_0_count > target_1_count:
            df = pd.concat([
                df[df[self.__target_column] == 0].sample(n = target_1_count, random_state = 1),
                df[df[self.__target_column] == 1]
            ], axis = 0)
        elif target_0_count < target_1_count:
            df = pd.concat([
                df[df[self.__target_column] == 0],
                df[df[self.__target_column] == 1].sample(n = target_0_count, random_state = 1)
            ], axis = 0)
        
        return df
    
    def transform(self, df):
        df_treatment_0 = df[df['treatment'] == 0].copy()
        df_treatment_1 = df[df['treatment'] == 1].copy()
        
        df_treatment_0 = self.__make_equal(df_treatment_0)
        df_treatment_1 = self.__make_equal(df_treatment_1)
        df = pd.concat([df_treatment_0, df_treatment_1], axis = 0)
        return df
        
    __call__ = transform

In [14]:
target_col = 'conversion'
train_us = DataUndersampler(target_column = target_col)(train) # undersampling the data
del train

In [15]:
train_us.shape

(49150, 14)

In [16]:
train_us.columns

Index(['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       'f11', 'treatment', 'conversion'],
      dtype='object')

In [17]:
share_pivot(df = train_us, index_cols = ['treatment', 'conversion'])

#  share
treatment conversion              
0         0            2466   0.05
          1            2466   0.05
1         0           22109   0.45
          1           22109   0.45

In [25]:
from sklearn.preprocessing import StandardScaler
target_col = 'conversion'
X_train = train_us.drop(columns = ['treatment', target_col])
treatment_train = train_us['treatment'] # keep the treatment index
y_train = train_us[target_col]
X_test = test.drop(columns = ['treatment', target_col])
treatment_test = test['treatment']
y_test = test[target_col]   

## standardization
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform (X_test)

In [19]:
def get_metrics(y_val, uplift_effect, treatment_val):
    no_effect_share = round(100 * np.sum(uplift_effect == 0) / len(uplift_effect), 2)
    positive_effect_share = round(100 * np.sum(uplift_effect > 0) / len(uplift_effect), 2)
    negative_effect_share = round(100 * np.sum(uplift_effect < 0) / len(uplift_effect), 2)
    print(f'Model predicts positive effect in conversion probability after treatment for {positive_effect_share}% of cases.')
    print(f'Model predicts negative effect in conversion probability after treatment for {negative_effect_share}% of cases.')
    print(f'Model predicts no effect in conversion probability after treatment for {no_effect_share}% of cases.')
    
    # 30%
    upliftk = uplift_at_k(
        y_true = y_val, 
        uplift = uplift_effect, 
        treatment = treatment_val, 
        strategy='by_group', 
        k = 0.3
    )
    
    upliftk_all = uplift_at_k(
        y_true = y_val, 
        uplift = uplift_effect, 
        treatment = treatment_val, 
        strategy = 'overall',
    )

    qini_coef = qini_auc_score(
        y_true = y_val, 
        uplift = uplift_effect,
        treatment = treatment_val
    )

    ## auuc, focus on this (normalized Area Under the Uplift Curve score)
    uplift_auc = uplift_auc_score(
        y_true = y_val, 
        uplift = uplift_effect,
        treatment = treatment_val
    )
    wau = weighted_average_uplift(y_true = y_val, uplift = uplift_effect,
                                  treatment = treatment_val, strategy = 'by_group')
    wau_all = weighted_average_uplift(y_true = y_val, uplift = uplift_effect,
                                  treatment = treatment_val, strategy = 'overall')

    print(f'uplift at top 30% by group: {upliftk:.3f} by overall: {upliftk_all:.3f}\n',
          f'Weighted average uplift by group: {wau:.3f} by overall: {wau_all:.3f}\n',
          f'AUUC by group: {uplift_auc}\n',
          f'AUQC by group: {qini_coef}\n')

# Modeling <a class="anchor" id="modeling"></a>

## Meta Learners <a class="anchor" id="metalearners"></a>

### T-Learners <a class="anchor" id="tlearners"></a>

Write by self

In [20]:
def train_group(group,model):
    #group = 0: control, group = 1: treatment
    model_trained = clone(model)
    model_trained.fit(X_train[treatment_train == group], y_train[treatment_train == group])
    score = model_trained.predict_proba(X_test)[:, 1] # return test score
    return model_trained,score

try to gridsearch the parameter

In [21]:
# Define a function to perform grid search and train the model
def train_and_tune(X_train, y_train, estimator, param_grid, cv=5):
    grid_search = GridSearchCV(estimator, param_grid, cv=cv, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    return grid_search.best_estimator_, grid_search.best_score_

# Model for tunning
### xgboost
# estimator = XGBClassifier(booster='gbtree',n_estimators=70,max_depth=3,
#                               learning_rate=.1,subsample=0.7,reg_lambda=5，n_jobs=-1)
# # Parameter grid for hyperparameter tuning
# param_grid = {
#     'max_depth': [3,4,5],
#     'n_estimators': [50,70,100]
# }

# ### decision tree
estimator = DecisionTreeClassifier(random_state=1)
# Parameter grid for hyperparameter tuning
param_grid = {
    'max_depth': [3,4,5],
    'min_samples_leaf': [ 15, 20,30],
    'min_samples_split': [20,30,40,60]
}

# ### lgbm
# estimator = LGBMClassifier(subsample=0.7,max_depth=4, 
#                                       learning_rate=0.001, n_estimators=300,n_jobs = -1,
# )

# param_grid = {
#     "subsample":[1,0.9,0.7],
#     'max_depth': [10, 20, 40, -1],
#     'n_estimators': [100, 300, 500]
# }


# NN
# estimator = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', 
#               solver='adam', alpha=0.01,learning_rate='constant')
# # Parameter grid for hyperparameter tuning
# param_grid = {
#     'hidden_layer_sizes': [(100,),(30,),(20,20)]
# }


# Split your data according to treatment
X_train_control = X_train[treatment_train == 0]
y_train_control = y_train[treatment_train == 0]
X_train_treatment = X_train[treatment_train == 1]
y_train_treatment = y_train[treatment_train == 1]

# # Train and tune for control group
# control_model, control_score = train_and_tune(X_train_control, y_train_control, estimator, param_grid)
# # Train and tune for treatment group
# treatment_model, treatment_score = train_and_tune(X_train_treatment, y_train_treatment,estimator,  param_grid)

# remember to record the best model
print(control_model)
print(treatment_model)

## control
# ct_model, ct_score = train_group(0,control_model)
ct_score = control_model.predict_proba(X_test)[:, 1]

## treatment
# tr_model, tr_score = train_group(1,treatment_model)
tr_score = treatment_model.predict_proba(X_test)[:, 1]

## metrics
uplift_effect = tr_score - ct_score
get_metrics(y_val = y_test, uplift_effect = uplift_effect, treatment_val = treatment_test)

NameError: name 'control_model' is not defined

In [100]:
## use the best model we can get
ct_model = LGBMClassifier(subsample=0.7,max_depth=4, 
                                       learning_rate=0.001, n_estimators=300,n_jobs = -1,
)
tr_model = LGBMClassifier(subsample=0.7,max_depth=4, 
                                       learning_rate=0.001, n_estimators=500,n_jobs = -1,
)

## control
ct_model, ct_score = train_group(0,ct_model)

## treatment
tr_model, tr_score = train_group(1,tr_model)

## metrics
uplift_effect = tr_score - ct_score
get_metrics(y_val = y_test, uplift_effect = uplift_effect, treatment_val = treatment_test)
## plot to see the uplift score
# plot_uplift_preds(trmnt_preds=tr_score, ctrl_preds=ct_score,log = True)

## auuc from causalml
df = pd.DataFrame({
    'model': uplift_effect,
    'y': y_test,
    'w': treatment_test
})
auuc = auuc_score(df, outcome_col='y', treatment_col='w',  normalize=True)
print(auuc)

Model predicts positive effect in conversion probability after treatment for 11.07% of cases.
Model predicts negative effect in conversion probability after treatment for 88.93% of cases.
Model predicts no effect in conversion probability after treatment for 0.0% of cases.
uplift at top 30% by group: 0.004 by overall: 0.003
 Weighted average uplift by group: 0.001 by overall: 0.001
 AUUC by group: 0.005620676868331139
 AUQC by group: 0.17538186464284125

model     0.871979
Random    0.497005
dtype: float64


## Methods not applied

In [ ]:
# ct_model = XGBClassifier(
#     random_state = 1,
#     n_jobs = -1,
#     eval_metric = 'logloss'
# )

# estimator = DecisionTreeClassifier(
#     random_state = 1, 
#     min_samples_leaf = 15
# )


# ul_model = TwoModels(clone(estimator), clone(estimator))
# ul_model.fit(X_train, treatment_train, y_train)
# uplift_effect = ul_model.predict(X_test)
# get_metrics(y_val = y_test, uplift_effect = uplift_effect, treatment_val = treatment_test)

In [ ]:
## solo
# estimator = DecisionTreeClassifier(max_depth=5, min_samples_leaf=30, min_samples_split=20,
#                        random_state=1)

# ul_model = SoloModel(estimator=estimator)
# ul_model.fit(X_train, treatment_train, y_train)
# uplift_effect = ul_model.predict(X_test)
# get_metrics(y_val = y_test, uplift_effect = uplift_effect, treatment_val = treatment_test)
# plot_uplift_preds(trmnt_preds=tr_score, ctrl_preds=ct_score,log = True)

In [ ]:
## Two dependent models
# ct_model = LGBMClassifier(learning_rate=0.001, max_depth=10, n_estimators=300, subsample=1)


# tr_estimator = LGBMClassifier(learning_rate=0.001, max_depth=10, n_estimators=500, subsample=1)

# ul_model = TwoModels(clone(ct_model), clone(tr_estimator),method='ddr_control')
# ul_model.fit(X_train, treatment_train, y_train)
# uplift_effect = ul_model.predict(X_test)
# get_metrics(y_val = y_test, uplift_effect = uplift_effect, treatment_val = treatment_test)
# plot_uplift_preds(trmnt_preds=ul_model.trmnt_preds_, ctrl_preds=ul_model.ctrl_preds_,log = True)

## ensembling

In [ ]:
# from sklearn.ensemble import StackingClassifier
# from sklearn.linear_model import LogisticRegression

# # Define the base models
# base_models = [
#     ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
#     ('xgb', XGBClassifier(n_estimators=100, random_state=42))
# ]

# # Define the meta-model
# meta_model = LogisticRegression()

# # Create the stacking ensemble
# stack_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)
# stack_model.fit(X_train, y_train_treatment, y_train_control)
# predicted_uplift = stack_model.predict(X_val)